In [30]:

# import imageio
# imageio.plugins.ffmpeg.download()

from importlib import reload
from moviepy.editor import VideoFileClip
from IPython.display import HTML

import numpy as np
import sys
import os

# Fix imports in Jupyter Notebooks:

sys.path.append("/Users/daniel.gamezfranco/Dropbox/! CARND/CarND-T1-P1-Lane-Lines/")


# Visualizations will be shown in the notebook:

%matplotlib inline


# Import local deps

import src.helpers.lineRegressorWithMemory as LRM
import src.helpers.imageProcessing as IP
import src.helpers.plot as P

# TODO: Needed? They should be always there because they have just been imported!

if 'src.helpers.imageProcessing' in sys.modules:
    print("Reloading Modules...")
    reload(LRM)
    reload(IP)
    reload(P)
    print("Modules Reloaded!")


Reloading Modules...
Modules Reloaded!


In [34]:

# IMAGE PROCESSING:

rightLineStream = LRM.LineStream()
leftLineStream = LRM.LineStream()

def process_image(imageRGB):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    imageHLS = IP.rgb2hls(imageRGB)
    imageYW = IP.yellowAndWhiteHlsFilter(imageHLS)
    
    WIDTH = imageYW.shape[1]
    HEIGHT = imageYW.shape[0]
    
    REGION_MASK = np.array([[
        (0.05 * WIDTH, 1.00 * HEIGHT),
        (0.45 * WIDTH, 0.60 * HEIGHT),
        (0.55 * WIDTH, 0.60 * HEIGHT),
        (0.95 * WIDTH, 1.00 * HEIGHT)
    ]], dtype=np.int32)
    
    imageRegion = IP.region_of_interest(imageYW, REGION_MASK)
    imageBlur = IP.gaussian_blur(imageRegion, 5)
    imageCanny = IP.canny(imageBlur, 50, 100)
    imagesHoughLines = IP.hough_lines(imageCanny, 2, np.pi / 180, 30, 20, 80)
        
    imageLines = np.zeros((imageRGB.shape[0], imageRGB.shape[1], 3), dtype=np.uint8)
    
    if imagesHoughLines is None:
        rightLineStream.drawLineFromPoints(imageLines, [[]], [], [], [255, 0, 0])
        leftLineStream.drawLineFromPoints(imageLines, [[]], [], [], [255, 0, 0])
    else:
        linesPoints = IP.classifyLinesPoints(imagesHoughLines)
        right = linesPoints['right']
        left = linesPoints['left']
        
        rightLineStream.drawLineFromPoints(imageLines, right['X'], right['Y'], right['weights'], [255, 0, 0])
        leftLineStream.drawLineFromPoints(imageLines, left['X'], left['Y'], left['weights'], [255, 0, 0])
    
    # TODO: Maybe just return the line and draw it here
    
    return IP.weighted_img(imageLines, imageRGB)


# VIDEO PROCESSING:

outputDir = "./output/"
videosDir = "./assets/videos/"
videosNames = os.listdir(videosDir)

def process_video(videoName):
    print("\nProcessing %s:" % videoName)

    rightLineStream.reset()
    leftLineStream.reset()
    
    inputClip = VideoFileClip(videosDir + videoName)
    outputClip = inputClip.fl_image(process_image) # NOTE: this function expects color images!
    %time outputClip.write_videofile(outputDir + videoName, audio=False)

print("Available videos:\n")

for videoName in videosNames: print(videoName)
    

Available videos:

001 - Solid White Right.mp4
002 - Solid Yellow Left.mp4
003 - Challenge.mp4
004 - Advanced Project.mp4
005 - Advanced Project Challenge.mp4
006 - Advanced Project Harder Challenge.mp4


In [35]:

for videoName in videosNames:
    process_video(videoName)



Processing 001 - Solid White Right.mp4:
[MoviePy] >>>> Building video ./output/001 - Solid White Right.mp4


[MoviePy] Writing video ./output/001 - Solid White Right.mp4


  0%|          | 0/222 [00:00<?, ?it/s]

  3%|▎         | 7/222 [00:00<00:03, 64.11it/s]

  5%|▌         | 12/222 [00:00<00:03, 57.62it/s]

...

100%|█████████▉| 1195/1200 [00:41<00:00, 32.22it/s]

100%|█████████▉| 1199/1200 [00:41<00:00, 32.55it/s]

[MoviePy] Done.


[MoviePy] >>>> Video ready: ./output/006 - Advanced Project Harder Challenge.mp4 



CPU times: user 43.3 s, sys: 8.12 s, total: 51.4 s
Wall time: 42.5 s


In [7]:

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


<video width="960" height="540" controls>
  <source src="white.mp4">
</video>